In [7]:
from StroblSimFuns import *

lambdas = 1.0
relevance = 0.2
shrink_modes = ["hs_entropy"]
n_replications = 1
clf_type="rf"
n_samples=100

if clf_type == "rf":
    hsc = ShrinkageClassifier(RandomForestClassifier(max_depth=max_depth))
elif clf_type == "dt":
    hsc = ShrinkageClassifier(DecisionTreeClassifier(max_depth=max_depth))

param_grid = {"shrink_mode": [shrink_mode], "lmb": lambdas}

SystemError: initialization of _internal failed without raising an exception

In [ ]:
#!pip install shap
#!pip install numba
from StroblSimFuns import *

In [ ]:
X, y = simulate_Strobl(n_samples, relevance)
X_test, y_test = simulate_Strobl(n_samples, relevance)

In [ ]:
lmb_scores = cross_val_shrinkage(
                hsc, X, y, param_grid, n_splits=5, score_fn = "AUC", 
                n_jobs=1, return_param_values=False)

In [ ]:
best_idx = np.argmax(lmb_scores)
best_lmb = lambdas[best_idx]
hsc.set_shrink_params(shrink_mode=shrink_modes[0], lmb=best_lmb)

In [ ]:
shap_vals = TreeExplainer(hsc.estimator_)